# LLMs with Hugging Face
In this notebook, we'll take a whirlwind tour of some top applications using Large Language Models (LLMs):

* Summarization
* Sentiment analysis
* Translation
* Zero-shot classification
* Few-shot learning

## Learning Objectives
1. Use a variety of existing models for a variety of common applications.
2. Understand basic prompt engineering.
3. Understand search vs. sampling for LLM inference.
4. Get familiar with the main Hugging Face abstractions: datasets, pipelines, tokenizers, and models.

Libraries : 

* sacremoses : for the translation model [Helsinki-NLP/opus-mt-en-es]
* for English -> Spanish